# Coursera Data Science Capstone Project
## Swift Key Predictive Text Analytics
## David Williams, PMP
## dww@pacificera.com

## Data Acquisition and Initial Analysis
Data for the final project was obtained from this [location](https://d396qusza40orc.cloudfront.net/dsscapstone/dataset/Coursera-SwiftKey.zip).  Our first step is to get some summary statistics on what we are dealing with.  We will limit our analysis to the US English files.

In [9]:
%%bash
ls -lh data/final/en_US/*
wc -l data/final/en_US/*

-rw-r--r--@ 1 David  staff   200M Jul 22  2014 data/final/en_US/en_US.blogs.txt
-rw-r--r--@ 1 David  staff   196M Jul 22  2014 data/final/en_US/en_US.news.txt
-rw-r--r--@ 1 David  staff   159M Jul 22  2014 data/final/en_US/en_US.twitter.txt
  899288 data/final/en_US/en_US.blogs.txt
 1010242 data/final/en_US/en_US.news.txt
 2360148 data/final/en_US/en_US.twitter.txt
 4269678 total


So the files are roughly 150M - 200M.  The course materials provide some strong hints that these files should be randomly sampled to reduce the size for the initial analysis.  I'm going to do this with some UNIX commands instead of writing code.  I'm going to limit the sample sizes to 1% of what the original data sets are.

In [1]:
%%bash
gshuf -n 23696 -o data/sample/en_US/en_US.twitter.txt data/final/en_US/en_US.twitter.txt
gshuf -n 10102 -o data/sample/en_US/en_US.news.txt data/final/en_US/en_US.news.txt
gshuf -n 8992 -o data/sample/en_US/en_US.blogs.txt data/final/en_US/en_US.blogs.txt

In [2]:
%%bash
ls -lh data/sample/en_US/*
wc -l data/sample/en_US/*

-rw-r--r--  1 David  staff   2.0M Mar 11 10:01 data/sample/en_US/en_US.blogs.txt
-rw-r--r--  1 David  staff   2.0M Mar 11 10:01 data/sample/en_US/en_US.news.txt
-rw-r--r--  1 David  staff   1.6M Mar 11 10:01 data/sample/en_US/en_US.twitter.txt
    8992 data/sample/en_US/en_US.blogs.txt
   10102 data/sample/en_US/en_US.news.txt
   23696 data/sample/en_US/en_US.twitter.txt
   42790 total


In [23]:
%%bash 
#Now for the first quiz, we need to produce some simple stats
# get the longest line lengths
gwc -L data/final/en_US/en_US.* 

    40833 data/final/en_US/en_US.blogs.txt
    11384 data/final/en_US/en_US.news.txt
      173 data/final/en_US/en_US.twitter.txt
    40833 total


In [22]:
%%bash
# hate to love ratio
grep "hate" data/final/en_US/en_US.twitter.txt | wc -l
grep "love" data/final/en_US/en_US.twitter.txt | wc -l



   22138
   90956


In [1]:
22138 / 90956

[1] 0.2433924

## Token Time
I'm going to install the openNLP package in R so that I can check out how it works and try to tokenize my data files.


In [4]:
install.packages("openNLP",repos='http://cran.us.r-project.org')

also installing the dependencies ‘NLP’, ‘openNLPdata’, ‘rJava’




The downloaded source packages are in
	‘/private/var/folders/_s/rz9jcs3j7g994n_tpcq3m63r0000gn/T/RtmpNqmjxf/downloaded_packages’


Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [8]:
install.packages("tm", repos='http://cran.us.r-project.org')

also installing the dependency ‘slam’




The downloaded source packages are in
	‘/private/var/folders/_s/rz9jcs3j7g994n_tpcq3m63r0000gn/T/RtmpNqmjxf/downloaded_packages’


Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [3]:
install.packages("doMC",repos='http://cran.us.r-project.org')



The downloaded source packages are in
	'/private/var/folders/_s/rz9jcs3j7g994n_tpcq3m63r0000gn/T/Rtmpc1GaJZ/downloaded_packages'


Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [5]:
install.packages("ggplot2",repos='http://cran.us.r-project.org')


The downloaded source packages are in
	'/private/var/folders/_s/rz9jcs3j7g994n_tpcq3m63r0000gn/T/Rtmpc1GaJZ/downloaded_packages'


Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [7]:
install.packages("dplyr",repos='http://cran.us.r-project.org')


The downloaded source packages are in
	'/private/var/folders/_s/rz9jcs3j7g994n_tpcq3m63r0000gn/T/Rtmpc1GaJZ/downloaded_packages'


Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


# Load Libraries

In [2]:
library(tm)

Loading required package: NLP


In [4]:
library(doMC)
registerDoMC(cores = 8)

Loading required package: foreach
Loading required package: iterators
Loading required package: parallel


In [6]:
library(ggplot2)


Attaching package: 'ggplot2'

The following object is masked from 'package:NLP':

    annotate



In [8]:
library(dplyr)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



# Load the documents with the tm library

In [9]:
c <- Corpus(DirSource("data/sample/en_US/"), readerControl = list(reader = readPlain, language = "en_US", load = TRUE))

In [10]:
c <- tm_map(c, stripWhitespace, lazy=TRUE)
c <- tm_map(c, content_transformer(tolower),lazy=TRUE)
c <- tm_map(c, removeWords, stopwords("english"),lazy=TRUE)
c <- tm_map(c, removePunctuation, lazy=TRUE)
c <- tm_map(c, removeNumbers,lazy=TRUE)

## Exploratory Data Analysis
Tasks to accomplish

### Exploratory analysis 
- perform a thorough exploratory analysis of the data, understanding the distribution of words and relationship between the words in the corpora.
Understand frequencies of words and word pairs 
- build figures and tables to understand variation in the frequencies of words and word pairs in the data.

### Questions to consider

Some words are more frequent than others - what are the distributions of word frequencies?
What are the frequencies of 2-grams and 3-grams in the dataset?
How many unique words do you need in a frequency sorted dictionary to cover 50% of all word instances in the language? 90%?
How do you evaluate how many of the words come from foreign languages?
Can you think of a way to increase the coverage -- identifying words that may not be in the corpora or using a smaller number of words in the dictionary to cover the same number of phrases?

In [11]:
dtm <- DocumentTermMatrix(c)
tdm <- TermDocumentMatrix(c)

In [12]:
dtm <- removeSparseTerms(dtm, 0.5)
sparse = as.data.frame(as.matrix(removeSparseTerms(dtm, 0.1)))
# str(sparse)
summary(sparse)
#spl = runif(length(crude)) < 0.7
#train = subset(sparse, spl)
#test = subset(sparse, !spl)
#findFreqTerms(dtm, 150)

     aaron         aba      abandoned     abandoning        abbey      
 Min.   : 3   Min.   :1   Min.   :4.0   Min.   :1.000   Min.   :1.000  
 1st Qu.: 5   1st Qu.:1   1st Qu.:4.5   1st Qu.:1.000   1st Qu.:1.500  
 Median : 7   Median :1   Median :5.0   Median :1.000   Median :2.000  
 Mean   : 9   Mean   :1   Mean   :6.0   Mean   :1.333   Mean   :2.667  
 3rd Qu.:12   3rd Qu.:1   3rd Qu.:7.0   3rd Qu.:1.500   3rd Qu.:3.500  
 Max.   :17   Max.   :1   Max.   :9.0   Max.   :2.000   Max.   :5.000  
  abbreviated      abc          abilities      ability           able    
 Min.   :1    Min.   :5.000   Min.   :3.0   Min.   : 7.00   Min.   : 50  
 1st Qu.:1    1st Qu.:5.500   1st Qu.:3.5   1st Qu.:20.50   1st Qu.: 74  
 Median :1    Median :6.000   Median :4.0   Median :34.00   Median : 98  
 Mean   :1    Mean   :5.667   Mean   :4.0   Mean   :26.67   Mean   :104  
 3rd Qu.:1    3rd Qu.:6.000   3rd Qu.:4.5   3rd Qu.:36.50   3rd Qu.:131  
 Max.   :1    Max.   :6.000   Max.   :5.0   Max.   :

In [13]:
sparse2 = as.data.frame(as.matrix(removeSparseTerms(tdm, 0.1)))
str(sparse2)
summary(sparse2)

'data.frame':	9677 obs. of  3 variables:
 $ en_US.blogs.txt  : num  3 1 5 1 1 1 5 5 34 164 ...
 $ en_US.news.txt   : num  17 1 9 1 2 1 6 4 39 98 ...
 $ en_US.twitter.txt: num  7 1 4 2 5 1 6 3 7 50 ...


 en_US.blogs.txt   en_US.news.txt    en_US.twitter.txt
 Min.   :   1.00   Min.   :   1.00   Min.   :   1.00  
 1st Qu.:   2.00   1st Qu.:   2.00   1st Qu.:   1.00  
 Median :   5.00   Median :   6.00   Median :   3.00  
 Mean   :  16.65   Mean   :  16.84   Mean   :  15.07  
 3rd Qu.:  13.00   3rd Qu.:  15.00   3rd Qu.:   9.00  
 Max.   :1777.00   Max.   :2520.00   Max.   :1536.00  

In [14]:
Docs(tdm)
nTerms(tdm)
findFreqTerms(tdm, 2,3)

[1] "en_US.blogs.txt"   "en_US.news.txt"    "en_US.twitter.txt"

[1] 58988

[1] "aaas"                     "aarp"                    
    [3] "aashika"                  "aba"                     
    [5] "abandonment"              "abba"                    
    [7] "abbott"                   "abbreviated"             
    [9] "abbys"                    "abcs"                    
   [11] "abdominals"               "abdul"                   
   [13] "aberdeen"                 "abide"                   
   [15] "abiding"                  "abigail"                 
   [17] "ablaze"                   "abnegation"              
   [19] "abomination"              "aboriginal"              
   [21] "abortive"                 "abouted"                 
   [23] "abruptly"                 "absences"                
   [25] "absinthe"                 "absorbent"               
   [27] "absorbing"                "abstinence"              
   [29] "abstraction"              "abu"                     
   [31] "abusers"                  "abysmal"                 
   [33] "academia"                 "academically"            
   [35] "acai"                     "accelerator"             
   [37] "accepts"                  "accessorized"            
   [39] "accidently"               "acclaimed"               
   [41] "accolades"                "accommodating"           
   [43] "accommodations"           "accompanies"             
   [45] "accomplishing"            "accordion"               
   [47] "accountants"              "accreditation"           
   [49] "accretive"                "acct"                    
   [51] "accusation"               "accustomed"              
   [53] "ached"                    "achievable"              
   [55] "acids"                    "ack"                     
   [57] "acknowledgement"          "acknowledges"            
   [59] "aclu"                     "acme"                    
   [61] "acoustics"                "acquaintance"            
   [63] "acquaintances"            "acquainted"              
   [65] "acquiring"                "acquittal"               
   [67] "acrobatics"               "acronyms"                
   [69] "acs"                      "acta"                    
   [71] "actin"                    "activate"                
   [73] "activated"                "activism"                
   [75] "actresses"                "acupuncture"             
   [77] "acute"                    "acutes"                  
   [79] "ada"                      "adage"                   
   [81] "adamant"                  "adames"                  
   [83] "adaptive"                 "addicting"               
   [85] "addictions"               "addis"                   
   [87] "addison"                  "adelaide"                
   [89] "adelson"                  "adept"                   
   [91] "adeptness"                "adhered"                 
   [93] "adhesive"                 "adhesives"               
   [95] "adjectives"               "adjoining"               
   [97] "adjourned"                "adjunct"                 
   [99] "adjustable"               "adler"                   
  [101] "administer"               "administering"           
  [103] "administratores"          "admirable"               
  [105] "admiral"                  "admirer"                 
  [107] "admirers"                 "admiring"                
  [109] "admissions"               "ado"                     
  [111] "adobe"                    "adolescent"              
  [113] "adolescents"              "adoptive"                
  [115] "adored"                   "adores"                  
  [117] "adrian"                   "adriana"                 
  [119] "adrianna"                 "adultery"                
  [121] "adv"                      "advent"                  
  [123] "adversaries"              "adversely"               
  [125] "advert"                   "advertisements"          
  [127] "advertisers"              "advil"                   
  [129] "advisable"                "

In [15]:
head(sparse2)
sparse[1,]

,en_US.blogs.txt,en_US.news.txt,en_US.twitter.txt
aaron,3,17,7
aba,1,1,1
abandoned,5,9,4
abandoning,1,1,2
abbey,1,2,5
abbreviated,1,1,1


ERROR: Error in sprintf(wrap, header, body): 'fmt' length exceeds maximal format length 8192


,aaron,aba,abandoned,abandoning,abbey,abbreviated,abc,abilities,ability,able,abnormal,abortion,abortions,about,above,abraham,abroad,absence,absent,absolutely,absorb,absorbed,abstract,absurd,abuse,abusing,abv,academic,academics,academy,accelerate,accent,accents,accept,acceptable,acceptance,accepted,accepting,access,accessible,accessories,accident,accidental,accidentally,accommodate,accompanied,accomplice,accomplish,accomplished,accomplishment,accomplishments,accord,according,accordingly,account,accountability,accountable,accounted,accounting,accounts,accuracy,accurate,accused,ace,ache,achieve,achieved,achievement,achievements,achieving,acid,acknowledge,acknowledged,acl,acoustic,acquaintances,acquire,acquired,acquisition,across,act,acted,acting,action,actions,active,actively,activities,activity,actor,actors,actress,acts,actual,actually,adam,adams,adapt,adaptation,adapted,adapting,add,added,addiction,addictive,adding,addison,addition,additional,address,addressed,addresses,addressing,adds,adele,adjust,adjustment,administration,administrative,administrators,admire,admission,admit,admitted,adopt,adopted,adoption,adorable,adrienne,ads,adult,adults,advance,advanced,advances,advancing,advantage,advantages,adventure,adventures,adventurous,adversity,advertising,advice,advise,advised,adviser,advisor,advocacy,advocates,aesthetic,affairs,affect,affected,affectionate,affects,affiliated,affinity,afford,affordable,afghan,afghanistan,aforementioned,afraid,africa,african,after,aftermath,afternoon,afterwards,again,against,age,aged,agencies,agency,agenda,agent,agents,ages,aggravated,aggregate,aggressive,ago,agonizing,agony,agree,agreed,agreement,agrees,ahead,ahem,aide,aim,aimed,aiming,aint,air,airconditioned,aircraft,aired,airing,airline,airlines,airplane,airport,airwaves,aisle,akin,akron,alabama,alameda,alan,alarm,alarming,alarms,alas,alaska,albeit,albert,album,albums,albuquerque,alcohol,alcoholism,ale,alejandro,alert,alerts,ales,alex,alexander,alexandria,alfred,ali,alice,alicia,alien,aliens,alike,alive,all,allaround,alleged,allegedly,allen,alley,alliance,alliances,allied,allow,allowed,allowing,allows,allstar,alltime,almonds,almost,alone,along,alongside,aloud,alpha,already,also,alter,altered,alternate,alternating,alternative,although,alto,altogether,aluminum,alumni,alvin,always,amanda,amazed,amazes,amazing,amazingly,amazon,amber,ambition,ambitious,ambulance,amendment,america,american,americans,americas,amid,amino,among,amount,amounts,amphitheater,amsterdam,amtrak,amusement,amy,ana,anal,analog,analysis,analyst,analyze,analyzing,anatomy,anchor,ancient,and,anderson,andor,andre,andrea,andrew,andrews,android,andy,angel,angeles,angelic,angelina,angels,anger,angle,angry,animal,animals,animation,ankle,ann,anne,annie,anniversary,announce,announced,announcement,announcements,announcing,annoying,annual,anonymous,another,answer,answered,answering,answers,anthem,anthony,antibiotics,anticipate,anticipation,antique,antonio,ants,anxiety,anxious,anxiously,any,anybody,anymore,anyone,anything,anytime,anyway,anywhere,apart,apartment,apiece,apollo,apologize,apology,app,apparel,apparently,appeal,appeals,appear,appearance,appeared,appearing,appears,appetites,appetizer,apple,apples,applicants,application,applications,applied,applies,apply,applying,appointment,appointments,appreciate,appreciated,appreciation,appreciative,apprentice,approach,approaches,approaching,appropriate,appropriately,approval,approve,approved,approves,approximately,apps,april,arbitrarily,arch,architect,architects,archives,arctic,are,area,areas,areed,arena,arent,argentina,arguably,argue,argues,argument,arguments,ariel,arizona,ark,arkansas,arm,armani,armed,arms,armstrong,army,arnold,aroma,around,arrange,arrangements,arranging,arrest,arrested,arrival,arrive,arrived,arrives,arriving,arrogant,arrow,arrows,arsenal,art,arteries,arthur,article,articles,articulate,artificial,artist,artistic,artistry,artists,arts,artwork,ash,ashes,ashland,ashley,asia,asian,asians,aside,ask,asked,asking,asks,asleep,asparagus,a

In [26]:
sum(sparse2[1,])
head(sum(sparse2[]))
rs <- as.data.frame(rowSums(sparse2[]))
names(rs) = c("freq")
rs$word <- rownames(rs)
str(rs)
#rss <- rs[order(rs$freq),]
rss <- arrange(rs, -freq)
head(rs)
head(rss)

[1] 27

[1] 469915

'data.frame':	9677 obs. of  2 variables:
 $ freq: num  27 3 18 4 8 3 17 12 80 312 ...
 $ word: chr  "aaron" "aba" "abandoned" "abandoning" ...


,freq,word
aaron,27,aaron
aba,3,aba
abandoned,18,abandoned
abandoning,4,abandoning
abbey,8,abbey
abbreviated,3,abbreviated


,freq,word
1,5202,the
2,3162,will
3,3020,just
4,2997,said
5,2903,one
6,2735,like


In [21]:
names(rs)

[1] "freq"

In [18]:
library(ggplot2)
qplot(sparse)

Don't know how to automatically pick scale for object of type data.frame. Defaulting to continuous.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
Warning message:
In is.na(x): is.na() applied to non-(list or vector) of type 'NULL'Warning message:
: Removed 1 rows containing missing values (position_stack).

ERROR: Error in matrix(value, n, p): 'data' must be of a vector type, was 'NULL'


ERROR: Error in file(con, "rb"): cannot open the connection


plot without title

In [31]:
findAssocs(tdm, "david",.9)

$david
          "demand             "opec           "opec's             "they 
             1.00              1.00              1.00              1.00 
          address         addressed         advantage           again," 
             1.00              1.00              1.00              1.00 
         although          analysis           analyst          anything 
             1.00              1.00              1.00              1.00 
      associates,             bijan             bpd."          brothers 
             1.00              1.00              1.00              1.00 
        cambridge            cannot            center             cera. 
             1.00              1.00              1.00              1.00 
    characterized          cheating            closer         condition 
             1.00              1.00              1.00              1.00 
          control          critical           cutting            daniel 
             1.00              1.00              1.00              1.00 
           deemed   differentials,"           dillard          director 
             1.00              1.00              1.00              1.00 
          earlier              easy            editor             eight 
             1.00              1.00              1.00              1.00 
      environment            excess          excesses           expects 
             1.00              1.00              1.00              1.00 
            faces           harvard      immediately.        initiative 
             1.00              1.00              1.00              1.00 
       interview.             issue              june             june, 
             1.00              1.00              1.00              1.00 
             keep             learn            lesson               ltd 
             1.00              1.00              1.00              1.00 
          manager          market's           mideast           mizrahi 
             1.00              1.00              1.00              1.00 
         mizrahi,           mlotok, moussavar-rahmani          movement 
             1.00              1.00              1.00              1.00 
           named.              need          optimism        optimistic 
             1.00              1.00              1.00              1.00 
     organization           outlook           output.              paul 
             1.00              1.00              1.00              1.00 
      pessimistic         principal           problem        problems," 
             1.00              1.00              1.00              1.00 
        problems.      production,"       production.          prompted 
             1.00              1.00              1.00              1.00 
          quarter           quotas,         readdress            regain 
             1.00              1.00              1.00              1.00 
         regional         reiterate          reports,           reuters 
             1.00              1.00              1.00              1.00 
           rising           salomon         scheduled            seeing 
             1.00              1.00              1.00              1.00 
          session        slackens,"             slide              soon 
             1.00              1.00              1.00              1.00 
             sort             spoke           spriggs            supply 
             1.00              1.00              1.00              1.00 
            teach         telephone            then,"          thought. 
             1.00              1.00              1.00              1.00 
         together              told               try         uncertain 
             1.00              1.00              1.00              1.00 
     university's          unlikely             wants             way," 
             1.00              1.00              1.00              1.00 
           wishes           yergin,             yor

In [27]:
data("crude")
tdm <- TermDocumentMatrix(crude)
findAssocs(tdm, c("oil", "opec", "xyz"), c(0.7, 0.75, 0.1))

$oil
     15.8   clearly      late    trying       who    winter  analysts      said 
     0.87      0.80      0.80      0.80      0.80      0.80      0.79      0.78 
  meeting     above emergency    market     fixed      that    prices agreement 
     0.77      0.76      0.75      0.75      0.73      0.73      0.72      0.71 
   buyers 
     0.70 

$opec
  meeting emergency      15.8  analysts    buyers     above      said   ability 
     0.88      0.87      0.85      0.85      0.83      0.82      0.82      0.80 
     they   prices. agreement 
     0.80      0.79      0.76 

$xyz
numeric(0)